<section class="section1"><h1>Lab week 9</h1>
<p>This builds directly on week 8.</p>
<section class="section2"><h2>Building robust functions</h2>
<p>Constructing any mathematical algorithm is both an end in itself and a tool for future use. Being able to solve one linear programming problem using the Simplex method is good. Being able to solve any (suitable) linear programming problem using a function implementing the Simplex method allows us to ask more complicated questions. For example, how sensitive is the optimal solution to different constraints? This may be hard to answer analytically, but by calling our Simplex method function with slightly different inputs we can explore this question numerically.</p>
<p>However, this <em>only</em> works if the function we are calling is robust. We know that Simplex "fails" on certain inputs (for example, when the problem is unbounded). If we accidentally pass in incorrect input (an unbounded problem, for example), or nonsense (the right inputs, but in the wrong order, for example), then we need our function to catch this error and inform us. If we don't, we might use nonsense output from the Simplex function as "true" results for our more complex questions.</p>
</section><section class="section2"><h2>Documentation as contract</h2>
</section></section>

In [ ]:
import numpy as np

In [ ]:
def simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs):
    """
    Simplex method

    Parameters
    ----------
    cost_coeffs : vector of float
        Cost coefficients (appear in objective function), length nvars
    rhs_coeffs : vector of float
        Coefficients on RHS of inequalities, length nvars
    lp_coeffs : array of float
        Coefficients on LHS of inequalities, size nvars x nvars

    Returns
    -------
    status : str
        "optimal" or "unbounded" depending on problem.
    z : float
        Minimized objective function.
    x : vector of float
        Optimized coefficients.
    """
    nvars = len(rhs_coeffs)
    tableau = np.zeros((1+nvars, 1+2*nvars))
    tableau[1:, 0] = rhs_coeffs
    tableau[0, 1:nvars+1] = cost_coeffs
    tableau[1:, 1:nvars+1] = lp_coeffs
    tableau[1:, nvars+1:] = np.identity(nvars)
    # Find negative cost coefficients
    negative_cost_idx = np.nonzero(tableau[0, 1:] < 0)[0]
    if len(negative_cost_idx) == 0:
        return "unbounded", np.inf, np.zeros(nvars)
    while len(negative_cost_idx) > 0:
        column = negative_cost_idx[0] + 1
        # Bland's algorithm
        positive_tableau_idx =  np.nonzero(tableau[1:, column] > 0)[0]
        if len(positive_tableau_idx) == 0:
            return "unbounded", np.inf, np.zeros(nvars)
        ratio = tableau[1:, 0] / tableau[1:, column]
        row = positive_tableau_idx[np.argmin(ratio[positive_tableau_idx])] + 1
        # Pivot
        tableau[row, :] = tableau[row, :] / tableau[row, column]
        for work_row in range(0, nvars+1):
            if row == work_row:
                continue
            scale = tableau[work_row, column] / tableau[row, column]
            tableau[work_row, :] = tableau[work_row, :] - scale * tableau[row, :]
        negative_cost_idx = np.nonzero(tableau[0, 1:] < 0)[0]
    # Check
    negative_cost_idx = np.nonzero(tableau[0, 1:] < 0)[0]
    z = -tableau[0, 0]
    x =  tableau[1:, 0]
    return "optimal", z, x

<p>Above is an implementation of the simplex method. For now, concentrate on the documentation. In particular, note that the inputs are <em>required</em> to be arrays, that the arrays are <em>required</em> to be floating point (real) numbers, and that the sizes of the arrays are <em>required</em> to match.</p>
<p>What happens if we don't match the requirements? Possibly nothing bad:</p>

In [ ]:
cost_coeffs = np.array([-1, -1])
rhs_coeffs = np.array([4, 3])
lp_coeffs = np.array([[2, -1], [1, 2]])

print(simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs))

In [ ]:
('optimal', -2.6, array([2.2, 0.4]))


<p>This looks like we did nothing wrong, but check the types of the numbers we passed in:</p>

In [ ]:
print(cost_coeffs.dtype)
print(rhs_coeffs.dtype)
print(lp_coeffs.dtype)

In [ ]:
int64
int64
int64


<p>We actually passed in integers rather than floats. In many (but not all!) cases this doesn't make a difference, and so failing to perfectly match the requirement is not a problem.</p>
<p>However, in other cases it can really matter:</p>

In [ ]:
costs_too_long = np.array([1.0, 2.0, 3.0])
print(simplex_method(costs_too_long, rhs_coeffs, lp_coeffs))

In [ ]:
---------------------------------------------------------------------------

ValueError                                Traceback (most recent call last)

<ipython-input-5-fd1557774531> in <module>
      1 costs_too_long = np.array([1.0, 2.0, 3.0])
----> 2 print(simplex_method(costs_too_long, rhs_coeffs, lp_coeffs))


<ipython-input-2-99868ecd8a46> in simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)
     24     tableau = np.zeros((1+nvars, 1+2*nvars))
     25     tableau[1:, 0] = rhs_coeffs
---> 26     tableau[0, 1:nvars+1] = cost_coeffs
     27     tableau[1:, 1:nvars+1] = lp_coeffs
     28     tableau[1:, nvars+1:] = np.identity(nvars)


ValueError: could not broadcast input array from shape (3) into shape (2)


<p>The sizes were inconsistent, compared to the assumptions that we made within the algorithm. This leads to an error.</p>
<p>The interpretation of this within Python is that <em>documentation is a contract</em>. The docstring of a function promises that <em>if</em> the inputs match the requirements, <em>then</em> the output will match the specifications. If the input doesn't meet requirements, then the function may <em>try</em> to produce sensible output, but may produce errors (or worse: it may produce output that looks plausible, but is wrong).</p>
<p>Not all programming languages are as permissive as Python. Some will insist that the input matches very specific requirements and refuse to even try to compute an answer if not.</p>
<p>Python uses what is called "duck-typing": if the input looks like a duck and quacks like a duck, it will be treated as a duck. In the first case above, integers (in this case) behave the same way as floats, and so can be treated as floats.</p>
<section class="section1"><h1>Trying and failing</h1>
<p>The flip side of duck typing and the permissive Python approach is that when errors occur, as in the second case above, they can be difficult to link to the problem that caused them, and the error message can be hard to interpret.</p>
<p>We can improve our function by <em>adding in our own error messages</em> to explicitly catch, and explain, the error when calling the function.</p>
<p>Let's extract the key part of the function, using the "incorrect" inputs:</p>
</section>

In [ ]:
cost_coeffs = np.array([1.0, 2.0, 3.0])
rhs_coeffs = np.array([4, 3])
lp_coeffs = np.array([[2, -1], [1, 2]])

nvars = len(rhs_coeffs)
tableau = np.zeros((1+nvars, 1+2*nvars))
tableau[1:, 0] = rhs_coeffs
tableau[0, 1:nvars+1] = cost_coeffs

In [ ]:
---------------------------------------------------------------------------

ValueError                                Traceback (most recent call last)

<ipython-input-6-413a3e940c5b> in <module>
      6 tableau = np.zeros((1+nvars, 1+2*nvars))
      7 tableau[1:, 0] = rhs_coeffs
----> 8 tableau[0, 1:nvars+1] = cost_coeffs


ValueError: could not broadcast input array from shape (3) into shape (2)


<p>We see that we are programmatically setting up the tableau to have a specific size based on the number of equations, which is given by the number of RHS coefficients, which is given by the length of <code>rhs_coeffs</code>. However, we have <em>assumed</em> that this size, <code>nvars</code>, matches the number of variables, and hence the number of cost coefficients. We have documented this in the docstring - it is an underlying assumption in this (limited) implementation - but have not checked that it is true. When it is not true, as with this input, we get the error as the three entries in <code>cost_coeffs</code> cannot be stuffed into the two entries in that column of the <code>tableau</code>.</p>
<p>This is, indeed, an error in the inputs: they do not match our contract. It is an error in the values of the variable <code>cost_coeffs</code>, so calling it a <code>ValueError</code> makes sense. However, the message - however correct it may be - does not immediately help us fix the problem. Now that we know what the problem is and where it comes from, we can do better, by <em>raising the error ourselves</em>.</p>
<p>Modify the code to <em>check the inputs match the contract</em>:</p>

In [ ]:
nvars = len(rhs_coeffs)
tableau = np.zeros((1+nvars, 1+2*nvars))
tableau[1:, 0] = rhs_coeffs
if len(cost_coeffs) != nvars:
    raise ValueError("The length of cost_coeffs must match the length of rhs_coeffs.")
tableau[0, 1:nvars+1] = cost_coeffs

In [ ]:
---------------------------------------------------------------------------

ValueError                                Traceback (most recent call last)

<ipython-input-7-fc3a19c2e27b> in <module>
      3 tableau[1:, 0] = rhs_coeffs
      4 if len(cost_coeffs) != nvars:
----> 5     raise ValueError("The length of cost_coeffs must match the length of rhs_coeffs.")
      6 tableau[0, 1:nvars+1] = cost_coeffs


ValueError: The length of cost_coeffs must match the length of rhs_coeffs.


<p>This produces the exact same error as before, but now the error message tells the user calling the function exactly what needs to be fixed.</p>
<p>We can use this to make our original function more robust. We can even modify the code so that we remove the need for a <code>status</code> flag. We can raise an exception if the problem is unbounded.</p>

In [ ]:
def simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs):
    """
    Simplex method

    Parameters
    ----------
    cost_coeffs : vector of float
        Cost coefficients (appear in objective function), length nvars
    rhs_coeffs : vector of float
        Coefficients on RHS of inequalities, length nvars
    lp_coeffs : array of float
        Coefficients on LHS of inequalities, size nvars x nvars

    Returns
    -------
    z : float
        Minimized objective function.
    x : vector of float
        Optimized coefficients.
    """
    nvars = len(rhs_coeffs)
    tableau = np.zeros((1+nvars, 1+2*nvars))
    tableau[1:, 0] = rhs_coeffs
    if len(cost_coeffs) != nvars:
        raise ValueError("The length of cost_coeffs must match the length of rhs_coeffs.")
    tableau[0, 1:nvars+1] = cost_coeffs
    if lp_coeffs.shape != (nvars, nvars):
        raise ValueError("The shape of lp_coeffs must be (n, n) to match the length (n) of rhs_coeffs.")
    tableau[1:, 1:nvars+1] = lp_coeffs
    tableau[1:, nvars+1:] = np.identity(nvars)
    # Find negative cost coefficients
    negative_cost_idx = np.nonzero(tableau[0, 1:] < 0)[0]
    if len(negative_cost_idx) == 0:
        raise ValueError("The problem is unbounded")
    while len(negative_cost_idx) > 0:
        column = negative_cost_idx[0] + 1
        # Bland's algorithm
        positive_tableau_idx =  np.nonzero(tableau[1:, column] > 0)[0]
        if len(positive_tableau_idx) == 0:
            raise ValueError("The problem is unbounded")
        ratio = tableau[1:, 0] / tableau[1:, column]
        row = positive_tableau_idx[np.argmin(ratio[positive_tableau_idx])] + 1
        # Pivot
        tableau[row, :] = tableau[row, :] / tableau[row, column]
        for work_row in range(0, nvars+1):
            if row == work_row:
                continue
            scale = tableau[work_row, column] / tableau[row, column]
            tableau[work_row, :] = tableau[work_row, :] - scale * tableau[row, :]
        negative_cost_idx = np.nonzero(tableau[0, 1:] < 0)[0]
    # Check
    negative_cost_idx = np.nonzero(tableau[0, 1:] < 0)[0]
    z = -tableau[0, 0]
    x =  tableau[1:, 0]
    return z, x

<p>We can check that this behaves as expected by putting in an unbounded problem:</p>

In [ ]:
cost_coeffs = np.array([-1, -1])
rhs_coeffs = np.array([-1, -2])
lp_coeffs = np.array([[-1, 1], [-1, -1]])
simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)

In [ ]:
---------------------------------------------------------------------------

ValueError                                Traceback (most recent call last)

<ipython-input-9-4ff0d23ee693> in <module>
      2 rhs_coeffs = np.array([-1, -2])
      3 lp_coeffs = np.array([[-1, 1], [-1, -1]])
----> 4 simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)


<ipython-input-8-432b99016f91> in simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)
     38         positive_tableau_idx =  np.nonzero(tableau[1:, column] > 0)[0]
     39         if len(positive_tableau_idx) == 0:
---> 40             raise ValueError("The problem is unbounded")
     41         ratio = tableau[1:, 0] / tableau[1:, column]
     42         row = positive_tableau_idx[np.argmin(ratio[positive_tableau_idx])] + 1


ValueError: The problem is unbounded


<p>We now run into the issue with making better error messages: it blocks exploration. If we wanted to call the simplex method on lots of inputs, and find which was the "best" result, we would be stopped by the error message as soon as any case went wrong. We can see this when trying to varying the first cost coefficient in the unbounded problem:</p>

In [ ]:
costs = np.linspace(2, 0, 10)
for cost in costs:
    cost_coeffs = np.array([cost, -1])
    print(simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs))

In [ ]:
(1.0, array([-1., -3.]))
(1.0, array([-1., -3.]))
(1.0, array([-1., -3.]))
(1.0, array([-1., -3.]))
(1.0, array([-1., -3.]))



---------------------------------------------------------------------------

ValueError                                Traceback (most recent call last)

<ipython-input-10-8bc09ddd2faf> in <module>
      2 for cost in costs:
      3     cost_coeffs = np.array([cost, -1])
----> 4     print(simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs))


<ipython-input-8-432b99016f91> in simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)
     38         positive_tableau_idx =  np.nonzero(tableau[1:, column] > 0)[0]
     39         if len(positive_tableau_idx) == 0:
---> 40             raise ValueError("The problem is unbounded")
     41         ratio = tableau[1:, 0] / tableau[1:, column]
     42         row = positive_tableau_idx[np.argmin(ratio[positive_tableau_idx])] + 1


ValueError: The problem is unbounded


<p>However, we can get around this problem programmatically. The idea is to <em>try</em> and do a calculation. If it works, good. If not, we can decide what to do, based on the error. The Python syntax looks very much like <code>if</code>/<code>else</code> statements:</p>

In [ ]:
cost_coeffs = np.array([-1, -1])
try:
    simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)
except ValueError:
    print("That input does not work")

In [ ]:
That input does not work


<p>This means we can complete the loop by ignoring cases where the algorithm is unbounded:</p>

In [ ]:
costs = np.linspace(2, 0, 10)
for cost in costs:
    cost_coeffs = np.array([cost, -1])
    try:
        print(cost, simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs))
    except ValueError:
        print("cost", cost, "leads to unbounded problem")

In [ ]:
2.0 (1.0, array([-1., -3.]))
1.7777777777777777 (1.0, array([-1., -3.]))
1.5555555555555556 (1.0, array([-1., -3.]))
1.3333333333333335 (1.0, array([-1., -3.]))
1.1111111111111112 (1.0, array([-1., -3.]))
cost 0.8888888888888888 leads to unbounded problem
cost 0.6666666666666667 leads to unbounded problem
cost 0.44444444444444464 leads to unbounded problem
cost 0.22222222222222232 leads to unbounded problem
cost 0.0 leads to unbounded problem


<p>There are many additional error types that can be caught and checked: one that often comes up in mathematical calculations is zero division:</p>

In [ ]:
1/0

In [ ]:
---------------------------------------------------------------------------

ZeroDivisionError                         Traceback (most recent call last)

<ipython-input-13-9e1622b385b6> in <module>
----> 1 1/0


ZeroDivisionError: division by zero


<p>Exactly the same steps can be used for that case:</p>

In [ ]:
xs = [2, 0, -3]
for x in xs:
    try:
        print("10 divide", x, "is", 10/x)
    except ZeroDivisionError:
        print("Cannot divide by zero")

In [ ]:
10 divide 2 is 5.0
Cannot divide by zero
10 divide -3 is -3.3333333333333335


<section class="section1"><h1>Testing</h1>
<p>We can make our code more robust but we need to trust that it is correct. To do this we want to test against simple cases where we know the solution and check that it is right.</p>
<p>Note that we do not want to do this once. We have already modified our function a few times. Each time we change something we may be introducing errors. So each time we change something we want to re-run our tests to check we have not introduced a problem when trying to fix another. That means we want to make the tests as easy to run as possible.</p>
<p>Let us first look at a "solved" problem, where the LP coefficients are in the form of the identity matrix, so the optimal values of the coefficients match the RHS coefficients:</p>
</section>

In [ ]:
cost_coeffs = np.array([-1, -1])
rhs_coeffs = np.array([1, 1])
lp_coeffs = np.array([[1, 0], [0, 1]])
print(simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs))

In [ ]:
(-2.0, array([1., 1.]))


<ipython-input-8-432b99016f91>:41: RuntimeWarning: divide by zero encountered in true_divide
  ratio = tableau[1:, 0] / tableau[1:, column]


<p>We knew in advance that we should have <span>${\bf x} = (1, 1)$</span> and so <span>$z = -2$</span>. So we could check for that:</p>

In [ ]:
correct_z = -2
correct_xs = np.array([1, 1])
z, xs = simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)
print("z correct?", z == correct_z)
print("xs correct?", xs == correct_xs)

In [ ]:
z correct? True
xs correct? [ True  True]


<ipython-input-8-432b99016f91>:41: RuntimeWarning: divide by zero encountered in true_divide
  ratio = tableau[1:, 0] / tableau[1:, column]


<p>For the coefficient case, we do not want to look at each coefficient: we want to know that they're all correct. <code>numpy</code> allows us to check this:</p>

In [ ]:
print("xs correct", np.all(xs == correct_xs))

In [ ]:
xs correct True


<p>We want to turn this test into a function, so that every time we change our <code>simplex_method</code> function we can test it with one line of code. In Python, there are standard conventions for this:</p>
<ol>
<li>The testing function has a name starting <code>test_</code>, and typically takes no arguments;</li>
<li>The testing function uses <code>assert</code> to check that it is giving correct results.</li>
</ol>
<p>Explicitly, the test above would be:</p>

In [ ]:
def test_simple():
    cost_coeffs = np.array([-1, -1])
    rhs_coeffs = np.array([1, 1])
    lp_coeffs = np.array([[1, 0], [0, 1]])
    correct_z = -2
    correct_xs = np.array([1, 1])
    z, xs = simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)
    assert z == correct_z, "Objective function should be -2"
    assert np.all(xs == correct_xs), "Coefficients should match RHS"

In [ ]:
test_simple()

In [ ]:
<ipython-input-8-432b99016f91>:41: RuntimeWarning: divide by zero encountered in true_divide
  ratio = tableau[1:, 0] / tableau[1:, column]


<p>This produces no output! This is what we want: if the tests pass everything is fine and we continue. If the tests fail, we want to see that. We can check that by writing a function specifically designed to fail:</p>

In [ ]:
def test_simple_fails():
    cost_coeffs = np.array([-1, -1])
    rhs_coeffs = np.array([1, 1])
    lp_coeffs = np.array([[1, 0], [0, 1]])
    correct_z = -1
    correct_xs = np.array([1, 1])
    z, xs = simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)
    assert z == correct_z, "Objective function should be -2, but we are comparing to -1"
    assert np.all(xs == correct_xs), "Coefficients should match RHS"

test_simple_fails()

In [ ]:
<ipython-input-8-432b99016f91>:41: RuntimeWarning: divide by zero encountered in true_divide
  ratio = tableau[1:, 0] / tableau[1:, column]



---------------------------------------------------------------------------

AssertionError                            Traceback (most recent call last)

<ipython-input-20-512710b47034> in <module>
      9     assert np.all(xs == correct_xs), "Coefficients should match RHS"
     10 
---> 11 test_simple_fails()


<ipython-input-20-512710b47034> in test_simple_fails()
      6     correct_xs = np.array([1, 1])
      7     z, xs = simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)
----> 8     assert z == correct_z, "Objective function should be -2, but we are comparing to -1"
      9     assert np.all(xs == correct_xs), "Coefficients should match RHS"
     10 


AssertionError: Objective function should be -2, but we are comparing to -1


<p>We can write as many tests as we like, but each should add something new: that way, if a test fails, it tells us something specific has changed in the function we are testing. Here is an example:</p>

In [ ]:
cost_coeffs = np.array([-1, -1])
rhs_coeffs = np.array([5, -2])
lp_coeffs = np.array([[7, -5], [5, 3]])

print(simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs))

In [ ]:
(0.6666666666666667, array([ 1.66666667, -0.66666667]))


<p>We see that the results are floats, not integers, with long decimal expansions (probably representing e.g. <span>$2/3$</span> and similar). Testing these with direct equalities is dangerous:</p>

In [ ]:
z, xs = simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)
print(z == 2/3)
print(xs == np.array([5/3, -2/3]))

In [ ]:
False
[False False]


<p>Instead, we want to check that the results are <em>close</em> to the expected solution.</p>

In [ ]:
print(np.allclose(z, 2/3))
print(np.allclose(xs, np.array([5/3, -2/3])))

In [ ]:
True
True


<p>We can then build this into a test:</p>

In [ ]:
def test_close():
    cost_coeffs = np.array([-1, -1])
    rhs_coeffs = np.array([5, -2])
    lp_coeffs = np.array([[7, -5], [5, 3]])
    correct_z = 2/3
    correct_xs = np.array([5/3, -2/3])
    z, xs = simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)
    assert np.allclose(z, correct_z), "Objective function should be 2/3"
    assert np.allclose(xs, correct_xs), "Coefficients should be 5/3, -2/3"

test_close()

<p>We now have two (useful) tests. Every time we modify <code>simplex_method</code> we can now check that we have not introduced a bug by running two lines of code.</p>
<section class="section4"><h4>Exercise</h4>
<p>Move these tests into a file along with your <code>simplex_method</code> function: call this <code>simplex.py</code>.</p>
<p>Make sure you have <a href="https://docs.pytest.org/en/6.2.x/"><code>pytest</code></a> installed (it should be part of Anaconda: in a terminal type <code>import pytest</code> to check).</p>
<p>Then <code>pytest</code> can be used to run all the tests. In a terminal, type</p>
</section>

In [ ]:
import pytest
pytest.main(['simplex.py'])

<p>The resulting screen output should look something like:</p>

In [ ]:
%%bash
============================= test session starts ==============================
...
collected 3 items

simplex.py .F.                                                           [100%]

=================================== FAILURES ===================================
______________________________ test_simple_fails _______________________________

    def test_simple_fails():
        cost_coeffs = np.array([-1, -1])
        rhs_coeffs = np.array([1, 1])
        lp_coeffs = np.array([[1, 0], [0, 1]])
        correct_z = -1
        correct_xs = np.array([1, 1])
        z, xs = simplex_method(cost_coeffs, rhs_coeffs, lp_coeffs)
>       assert z == correct_z, "Objective function should be -2, but we are comparing to -1"
E       AssertionError: Objective function should be -2, but we are comparing to -1

simplex.py:76: AssertionError
=============================== warnings summary ===============================
simplex.py::test_simple
simplex.py::test_simple_fails
  /Users/ih3/Desktop/MATH1058/simplex.py:43: RuntimeWarning: divide by zero encountered in true_divide
    ratio = tableau[1:, 0] / tableau[1:, column]

-- Docs: https://docs.pytest.org/en/stable/warnings.html
=========================== short test summary info ============================
FAILED simplex.py::test_simple_fails - AssertionError: Objective function sho...
=================== 1 failed, 2 passed, 2 warnings in 0.16s ====================

<p>This has run all the tests for us and summarized the results. Using a test runner like <code>pytest</code> is invaluable when working with large codes, as it means every change can be automatically checked to see it hasn't broken existing code.</p>